In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

plt.rcParams.update({
    "figure.figsize": (15,5)
})

In [8]:
pilatus = pd.read_csv("order111064/order_111064_data.txt", sep=";")
luzern = pd.read_csv("order111065/order_111065_data.txt", sep=";")

# Aufbereitung der Daten

In [9]:
def df_aufbereitung(dataframe, columnnames):


    #Änderung des Zeitformats von Zahlen zu Datetime
    dataframe['time'] = pd.to_datetime(dataframe['time'], format='%Y%m%d')
    dataframe['time'] = pd.to_datetime(dataframe['time'], format='%Y.%m.%d')

    #Neubenennung der Spaltennamen
    dataframe.columns = columnnames


    #Entfernung von nichtvorhandenen Messungen
    dataframe = dataframe.replace('-', np.nan)
    dataframe.dropna(inplace=True)


    #Anpassung des Datatypes der Niederschlagsmenge
    dataframe['Niederschlag Tagessumme [mm]'] = dataframe['Niederschlag Tagessumme [mm]'].astype('float')

    return dataframe



columnnames_pilatus = ['Station','Datum','Temperatur Tagesmaximum [°C]','Temperatur Tagesminimum [°C]','Temperatur Tagesmittel [°C]','Niederschlag Tagessumme [mm]']
columnnames_luzern  = ['Station','Datum','Temperatur Tagesmaximum [°C]','Temperatur Tagesminimum [°C]','Temperatur Tagesmittel [°C]',"Neuschneehöhe Tagessumme [cm]",'Niederschlag Tagessumme [mm]']

pilatus = df_aufbereitung(pilatus, columnnames_pilatus)
luzern = df_aufbereitung(luzern, columnnames_luzern)


luzern['Neuschneehöhe Tagessumme [cm]'] = luzern['Neuschneehöhe Tagessumme [cm]'].astype('float')



## Schneefall

Als Faustregel gilt:

$$Niederschlag * 10 = Schneehöhe$$

Dazu muss die Tagesmitteltemperatur $\leq0$ sein.

In [10]:
pilatus['Neuschneehöhe Tagessumme [cm]']  = np.where(pilatus['Temperatur Tagesmittel [°C]']<=0, pilatus['Niederschlag Tagessumme [mm]'],0)

In [11]:
pilatus

,Station,Datum,Temperatur Tagesmaximum [°C],Temperatur Tagesminimum [°C],Temperatur Tagesmittel [°C],Niederschlag Tagessumme [mm],Neuschneehöhe Tagessumme [cm]
0,PIL,1981-01-01,-0.9,-11.4,-7.4,11.3,11.3
1,PIL,1981-01-02,-4.2,-11.8,-8.9,30.3,30.3
2,PIL,1981-01-03,0.5,-7.1,-3.4,40.8,40.8
3,PIL,1981-01-04,-1.6,-13.4,-6.2,65.3,65.3
4,PIL,1981-01-05,-11.0,-13.2,-12.2,0.5,0.5
...,...,...,...,...,...,...,...
15426,PIL,2023-03-28,1.8,-11.0,-5.1,1.3,1.3
15427,PIL,2023-03-29,4.9,-2.5,2.1,0.9,0.0
15428,PIL,2023-03-30,6.1,0.2,2.7,1.3,0.0
15429,PIL,2023-03-31,2.8,-3.8,-0.7,13.2,13.2


# Interaktiver Plot

In [12]:
import plotly.express as px

fig = px.bar(pilatus, x='Datum', y='Neuschneehöhe Tagessumme [cm]', color_discrete_sequence=['red'])
fig.add_bar(x=luzern['Datum'], y=luzern['Neuschneehöhe Tagessumme [cm]'], name='Wetterstation Luzern',marker={'color':'green'} )
fig.update_layout(barmode='overlay')
fig.show()
